In [1]:
import os
import numpy as np
import pandas as pd
from biopandas.pdb import PandasPdb
from biopandas.mmcif import PandasMmcif
import torch
import dgl
import tokenizers
import transformers
import os
from sklearn.preprocessing import MinMaxScaler
from transformers import AutoTokenizer, AutoModel, AutoConfig
# 分层特征处理架构（代码示例）
import numpy as np


def load_aaindex1(file_path):
    aaindex1_df = pd.read_csv(file_path, index_col='Description')
    aaindex_dict = {aa: aaindex1_df[aa].values for aa in aaindex1_df.columns}
    return aaindex_dict

def extract_features(sequence, aaindex_dict):
    features = []
    for aa in sequence:
        if aa in aaindex_dict:
            features.append(aaindex_dict[aa])
        else:
            features.append(np.full((len(next(iter(aaindex_dict.values()))),), np.nan))
    return np.array(features)

def generate_graph(coords, threshold=8.0):
    all_diffs = np.expand_dims(coords, axis=1) - np.expand_dims(coords, axis=0)
    distance = np.sqrt(np.sum(np.power(all_diffs, 2), axis=-1))
    adj = distance < threshold
    u, v = np.nonzero(adj)
    u, v = torch.from_numpy(u), torch.from_numpy(v)
    graph = dgl.graph((u, v), num_nodes=coords.shape[0])
    return graph

#model = "/root/autodl-tmp/ToxGIN-main/ESM"
model = "facebook/esm2_t36_3B_UR50D"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

tokenizer = AutoTokenizer.from_pretrained(model)
config = AutoConfig.from_pretrained(model, output_hidden_states=True)
config.hidden_dropout = 0.
config.hidden_dropout_prob = 0.
config.attention_dropout = 0.
config.attention_probs_dropout_prob = 0.
encoder = AutoModel.from_pretrained(model, config=config).to(device).eval()
print("model loaded")

def seq_encode(seq):
    spaced_seq = " ".join(list(seq))
    inputs = tokenizer.encode_plus(
        spaced_seq, 
        return_tensors=None, 
        add_special_tokens=True,
        max_length=60,
        padding=True,
        truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = encoder(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    last_hidden_states = outputs[0]
    encoded_seq = last_hidden_states[inputs['attention_mask'].bool()][1:-1]
    return encoded_seq

def aggre(s):
    if type(s.values[0]) == str:
        return s.values[0]
    return np.mean(s)

aa_map = {'VAL': 'V', 'PRO': 'P', 'ASN': 'N', 'GLU': 'E', 'ASP': 'D', 'ALA': 'A', 'THR': 'T', 'SER': 'S',
          'LEU': 'L', 'LYS': 'K', 'GLY': 'G', 'GLN': 'Q', 'ILE': 'I', 'PHE': 'F', 'CYS': 'C', 'TRP': 'W',
          'ARG': 'R', 'TYR': 'Y', 'HIS': 'H', 'MET': 'M'}



cuda:0


2025-05-03 10:11:19.225512: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-03 10:11:19.269698: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-03 10:11:19.930238: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/root/miniconda3/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still ch

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t36_3B_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model loaded


In [2]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
#压缩模块,可选/调参数
def compress(encode,features):
    variances = np.var(encode, axis=0)
    keep_dim = int(encode.shape[1] * 0.01)
    top_idx = np.argsort(variances)[-keep_dim:][::-1]
    preserved_feat = encode[:, top_idx]    
    residual_feat = np.delete(encode, top_idx, axis=1)
    scaler = RobustScaler(unit_variance=True)
    scaled_residual = scaler.fit_transform(residual_feat)    
    safe_dim = min(scaled_residual.shape[0]-1, scaled_residual.shape[1])
    pca = PCA(n_components=min(0.5, safe_dim), svd_solver='full')
    compressed_residual = pca.fit_transform(scaled_residual)
    esm_compressed = np.hstack([preserved_feat, compressed_residual])
    esm_var = np.var(esm_compressed, axis=0)
    top100_idx = np.argsort(esm_var)[-50:][::-1]
    esm_compressed = esm_compressed[:, top100_idx]

    
    chosen_lis = [22,23,108,111,132,142,143,144,146,147,151,163,398]
    # var_selector = VarianceThreshold(4)
    # aaindex_filtered = var_selector.fit_transform(features)
    # aaindex_var = np.var(aaindex_filtered, axis=0)
    # top100_idx = np.argsort(aaindex_var)[-100:][::-1]
    # aaindex_compressed = aaindex_filtered[:, top100_idx]
    if esm_compressed.shape[1]+aaindex_compressed.shape[1]<150:
        print(esm_compressed.shape,aaindex_compressed.shape)
    return esm_compressed,aaindex_compressed

In [2]:
import os
from pathlib import Path
import dgl


# 确保输出目录存在
df = pd.read_csv('train_sequence.csv')
root_dir = 'train_data'
os.makedirs(root_dir, exist_ok=True)
graphs = []
pdb = None

aaindex_dict = load_aaindex1('aaindex1.csv')
scaler = MinMaxScaler()
graph_dir = Path(root_dir) / 'dgl_graphs'
graph_dir.mkdir(exist_ok=True, parents=True)

valid_indices = []  # 记录有效处理的索引，后续可能需要更新数据框
train_data = {
    'target': df['label'].tolist(),
    'atoms': [],
    'coordinates': [],
    'features': [],
    'res_coords':[]
}
for index, row in df.iterrows():
    sequence = row.sequence
    pdb_file = f'train_structures/{sequence}.pdb'
    
    # 检查PDB文件是否存在
    if not os.path.exists(pdb_file):
        print(f"PDB文件 {pdb_file} 不存在，跳过样本 {sequence}")
        continue
    
    try:
        # 读取PDB文件
        atom_df = PandasPdb().read_pdb(pdb_file).df['ATOM']

        residue_df = atom_df.groupby('residue_number', as_index=False).agg(aggre).sort_values('residue_number')     
        # 生成残基名称序列
        residue_df['letter'] = residue_df.residue_name.map(aa_map)
        pdb_seq = ''.join(residue_df.letter.values)
        atom_df = atom_df.merge(residue_df[['residue_number', 'letter']],
                                on='residue_number', how='left')        
        # 验证生成的序列是否与预期一致
        if pdb_seq != sequence:
            print(f"序列不匹配: 文件{pdb_file}生成序列 {pdb_seq} 不等于预期序列 {sequence}")
            continue
        
        # 生成分子图
        res_coords = residue_df[['x_coord', 'y_coord', 'z_coord']].values
        graph = generate_graph(res_coords)
        
        # 验证图节点数等于序列长度
        if graph.number_of_nodes() != len(pdb_seq):
            print(f"图节点数{graph.num_nodes()} 与序列长度{len(pdb_seq)}不一致")
            continue
        graphs.append(graph) 
            
        # 获取原子信息和坐标
        atom_types = atom_df['element_symbol'].tolist()  # 原子类型（C, N, O 等）
        coords = atom_df[['x_coord', 'y_coord', 'z_coord']].values  # 原子坐标
        # 存储原子和坐标信息
        train_data['atoms'].append(atom_types)
        train_data['coordinates'].append(coords)
        # 生成特征向量
        encoded_pdb_seq = seq_encode(pdb_seq).cpu().numpy()
        features = extract_features(pdb_seq, aaindex_dict)
        # encoded_pdb_seq,features = compress(encoded_pdb_seq,features)
        # normalized_encoded = scaler.fit_transform(encoded_pdb_seq.astype(np.float32))
        # normalized_features = scaler.fit_transform(features.astype(np.float32))
        combined_features = np.concatenate([encoded_pdb_seq, features], axis=-1)

        # 保存特征文件
        npz_path = os.path.join(root_dir, f"{sequence}.npz")
        np.savez_compressed(npz_path, wildtype_seq=combined_features)
        
        # 保存图文件
        graph_path = os.path.join(graph_dir, f"{sequence}.bin")
        dgl.save_graphs(graph_path, [graph])
        
        valid_indices.append(index)
        feature = [];coord_lis=[];index=0
        for letter in atom_df['letter']:
            if letter!=pdb_seq[index]:
                index += 1
            feature.append(combined_features[index])
            coord_lis.append(res_coords[index])
        if len(feature)!=len(atom_df):
            print(len(feature),len(atom_df))
            break
        train_data['features'].append(feature)
        train_data['res_coords'].append(coord_lis)
        del atom_df
    except Exception as e:
        print(f"处理样本 {sequence} 时发生错误: {str(e)}")
        continue
dgl.save_graphs(root_dir+'/dgl_graph.bin', graphs)
# 可选: 更新数据框，仅保留有效样本
df_valid = df.loc[valid_indices].reset_index(drop=True)


In [2]:
import os
from pathlib import Path
import dgl


# 确保输出目录存在
df = pd.read_csv('test_sequence.csv')
root_dir = 'test_data'
os.makedirs(root_dir, exist_ok=True)
graphs = []
pdb = None

aaindex_dict = load_aaindex1('aaindex1.csv')
scaler = MinMaxScaler()
graph_dir = Path(root_dir) / 'dgl_graphs'
graph_dir.mkdir(exist_ok=True, parents=True)

valid_indices = []  # 记录有效处理的索引，后续可能需要更新数据框
test_data = {
    'target': df['label'].tolist(),
    'atoms': [],
    'coordinates': [],
    'features': [],
    'res_coords':[]
}
for index, row in df.iterrows():
    sequence = row.sequence
    pdb_file = f'test_structures/{sequence}.pdb'
    
    # 检查PDB文件是否存在
    if not os.path.exists(pdb_file):
        print(f"PDB文件 {pdb_file} 不存在，跳过样本 {sequence}")
        continue
    
    try:
        # 读取PDB文件
        atom_df = PandasPdb().read_pdb(pdb_file).df['ATOM']
        residue_df = atom_df.groupby('residue_number', as_index=False).agg(aggre).sort_values('residue_number')
        
        
        # 生成残基名称序列
        residue_df['letter'] = residue_df.residue_name.map(aa_map)
        pdb_seq = ''.join(residue_df.letter.values)
        atom_df = atom_df.merge(residue_df[['residue_number', 'letter']],
                                on='residue_number', how='left')         
        # 验证生成的序列是否与预期一致
        if pdb_seq != sequence:
            print(f"序列不匹配: 文件{pdb_file}生成序列 {pdb_seq} 不等于预期序列 {sequence}")
            continue
        
        # 生成分子图
        res_coords = residue_df[['x_coord', 'y_coord', 'z_coord']].values
        graph = generate_graph(res_coords)
        
        # 验证图节点数等于序列长度
        if graph.number_of_nodes() != len(pdb_seq):
            print(f"图节点数{graph.num_nodes()} 与序列长度{len(pdb_seq)}不一致")
            continue
        graphs.append(graph)     
        # 获取原子信息和坐标
        atom_types = atom_df['element_symbol'].tolist()  # 原子类型（C, N, O 等）
        coords = atom_df[['x_coord', 'y_coord', 'z_coord']].values  # 原子坐标
        # 存储原子和坐标信息
        test_data['atoms'].append(atom_types)
        test_data['coordinates'].append(coords)
        # 生成特征向量
        encoded_pdb_seq = seq_encode(pdb_seq).cpu().numpy()
        features = extract_features(pdb_seq, aaindex_dict)
        # encoded_pdb_seq,features = compress(encoded_pdb_seq,features)
        # normalized_encoded = scaler.fit_transform(encoded_pdb_seq.astype(np.float32))
        # normalized_features = scaler.fit_transform(features.astype(np.float32))
        combined_features = np.concatenate([encoded_pdb_seq, features], axis=-1)
        
        # 保存特征文件
        npz_path = os.path.join(root_dir, f"{sequence}.npz")
        np.savez_compressed(npz_path, wildtype_seq=combined_features)
        
        # 保存图文件
        graph_path = os.path.join(graph_dir, f"{sequence}.bin")
        dgl.save_graphs(graph_path, [graph])
        valid_indices.append(index)
        feature = [];coord_lis=[];index=0
        for letter in atom_df['letter']:
            if letter!=pdb_seq[index]:
                index += 1
            feature.append(combined_features[index])
            coord_lis.append(res_coords[index])
        if len(feature)!=len(atom_df):
            print(len(feature),len(atom_df))
            break
        test_data['features'].append(feature)
        test_data['res_coords'].append(coord_lis)
    except Exception as e:
        print(f"处理样本 {sequence} 时发生错误: {str(e)}")
        continue
dgl.save_graphs(root_dir+'/dgl_graph.bin', graphs)
# 可选: 更新数据框，仅保留有效样本
df_valid = df.loc[valid_indices].reset_index(drop=True)


In [ ]:
import os
import pandas as pd
from pathlib import Path
import dgl


# 确保输出目录存在
df = pd.read_csv('tamper_data.csv')
root_dir = 'tamper_data'
os.makedirs(root_dir, exist_ok=True)
graphs = []
pdb = None

aaindex_dict = load_aaindex1('aaindex1.csv')
scaler = MinMaxScaler()
graph_dir = Path(root_dir) / 'dgl_graphs'
graph_dir.mkdir(exist_ok=True, parents=True)

valid_indices = []  # 记录有效处理的索引，后续可能需要更新数据框
test_data = {
    'target': df['label'].tolist(),
    'atoms': [],
    'coordinates': [],
    'features': [],
    'res_coords':[]
}
for index, row in df.iterrows():
    sequence = row.sequence
    pdb_file = f'tamper_data/{sequence}.pdb'
    
    # 检查PDB文件是否存在
    if not os.path.exists(pdb_file):
        print(f"PDB文件 {pdb_file} 不存在，跳过样本 {sequence}")
        continue
    
    try:
        # 读取PDB文件
        atom_df = PandasPdb().read_pdb(pdb_file).df['ATOM']
        residue_df = atom_df.groupby('residue_number', as_index=False).agg(aggre).sort_values('residue_number')
        
        
        # 生成残基名称序列
        residue_df['letter'] = residue_df.residue_name.map(aa_map)
        pdb_seq = ''.join(residue_df.letter.values)
        atom_df = atom_df.merge(residue_df[['residue_number', 'letter']],
                                on='residue_number', how='left')         
        # 验证生成的序列是否与预期一致
        if pdb_seq != sequence:
            print(f"序列不匹配: 文件{pdb_file}生成序列 {pdb_seq} 不等于预期序列 {sequence}")
            continue
        
        # 生成分子图
        res_coords = residue_df[['x_coord', 'y_coord', 'z_coord']].values
        graph = generate_graph(res_coords)
        
        # 验证图节点数等于序列长度
        if graph.number_of_nodes() != len(pdb_seq):
            print(f"图节点数{graph.num_nodes()} 与序列长度{len(pdb_seq)}不一致")
            continue
        graphs.append(graph)     
        # 获取原子信息和坐标
        atom_types = atom_df['element_symbol'].tolist()  # 原子类型（C, N, O 等）
        coords = atom_df[['x_coord', 'y_coord', 'z_coord']].values  # 原子坐标
        # 存储原子和坐标信息
        test_data['atoms'].append(atom_types)
        test_data['coordinates'].append(coords)
        # 生成特征向量
        encoded_pdb_seq = seq_encode(pdb_seq).cpu().numpy()
        features = extract_features(pdb_seq, aaindex_dict)
        # encoded_pdb_seq,features = compress(encoded_pdb_seq,features)
        # normalized_encoded = scaler.fit_transform(encoded_pdb_seq.astype(np.float32))
        # normalized_features = scaler.fit_transform(features.astype(np.float32))
        print(encoded_pdb_seq.shape,features.shape)
        combined_features = np.concatenate([encoded_pdb_seq, features], axis=-1)
        
        # 保存特征文件
        npz_path = os.path.join(root_dir, f"{sequence}.npz")
        np.savez_compressed(npz_path, wildtype_seq=combined_features)
        
        # 保存图文件
        graph_path = os.path.join(graph_dir, f"{sequence}.bin")
        dgl.save_graphs(graph_path, [graph])
        valid_indices.append(index)
        feature = [];coord_lis=[];index=0
        for letter in atom_df['letter']:
            if letter!=pdb_seq[index]:
                index += 1
            feature.append(combined_features[index])
            coord_lis.append(res_coords[index])
        if len(feature)!=len(atom_df):
            print(len(feature),len(atom_df))
            break
        test_data['features'].append(feature)
        test_data['res_coords'].append(coord_lis)
    except Exception as e:
        print(f"处理样本 {sequence} 时发生错误: {str(e)}")
        continue
dgl.save_graphs(root_dir+'/dgl_graph.bin', graphs)
# 可选: 更新数据框，仅保留有效样本
df_valid = df.loc[valid_indices].reset_index(drop=True)


(15, 2560) (15, 553)
(13, 2560) (13, 553)
(11, 2560) (11, 553)
(15, 2560) (15, 553)
(6, 2560) (6, 553)
(8, 2560) (8, 553)
(10, 2560) (10, 553)
(15, 2560) (15, 553)
(14, 2560) (14, 553)
(13, 2560) (13, 553)
(14, 2560) (14, 553)
(14, 2560) (14, 553)
(12, 2560) (12, 553)
(13, 2560) (13, 553)
(13, 2560) (13, 553)
(13, 2560) (13, 553)
(11, 2560) (11, 553)
(14, 2560) (14, 553)
(13, 2560) (13, 553)
(11, 2560) (11, 553)
(11, 2560) (11, 553)
(11, 2560) (11, 553)
(11, 2560) (11, 553)
(15, 2560) (15, 553)
(11, 2560) (11, 553)
(13, 2560) (13, 553)
(13, 2560) (13, 553)
(13, 2560) (13, 553)
(15, 2560) (15, 553)
(15, 2560) (15, 553)
(14, 2560) (14, 553)
(12, 2560) (12, 553)
(13, 2560) (13, 553)
(13, 2560) (13, 553)
(13, 2560) (13, 553)
(12, 2560) (12, 553)
(9, 2560) (9, 553)
(7, 2560) (7, 553)
(13, 2560) (13, 553)
(13, 2560) (13, 553)
(13, 2560) (13, 553)
(13, 2560) (13, 553)
(14, 2560) (14, 553)
(13, 2560) (13, 553)
(14, 2560) (14, 553)
(14, 2560) (14, 553)
(14, 2560) (14, 553)
(14, 2560) (14, 553)


In [ ]:
import os
from pathlib import Path
import dgl


# 确保输出目录存在
df = pd.read_csv('test_sequence.csv')
root_dir = 'test_data'
os.makedirs(root_dir, exist_ok=True)
graphs = []
pdb = None

aaindex_dict = load_aaindex1('aaindex1.csv')
scaler = MinMaxScaler()
graph_dir = Path(root_dir) / 'dgl_graphs'
graph_dir.mkdir(exist_ok=True, parents=True)

valid_indices = []  # 记录有效处理的索引，后续可能需要更新数据框
test_data = {
    'target': df['label'].tolist(),
    'atoms': [],
    'coordinates': [],
    'features': [],
    'res_coords':[]
}
for index, row in df.iterrows():
    sequence = row.sequence
    pdb_file = f'test_structures/{sequence}.pdb'
    
    # 检查PDB文件是否存在
    if not os.path.exists(pdb_file):
        print(f"PDB文件 {pdb_file} 不存在，跳过样本 {sequence}")
        continue
    
    try:
        # 读取PDB文件
        atom_df = PandasPdb().read_pdb(pdb_file).df['ATOM']
        residue_df = atom_df.groupby('residue_number', as_index=False).agg(aggre).sort_values('residue_number')
        
        
        # 生成残基名称序列
        residue_df['letter'] = residue_df.residue_name.map(aa_map)
        pdb_seq = ''.join(residue_df.letter.values)
        atom_df = atom_df.merge(residue_df[['residue_number', 'letter']],
                                on='residue_number', how='left')         
        # 验证生成的序列是否与预期一致
        if pdb_seq != sequence:
            print(f"序列不匹配: 文件{pdb_file}生成序列 {pdb_seq} 不等于预期序列 {sequence}")
            continue
        
        # 生成分子图
        res_coords = residue_df[['x_coord', 'y_coord', 'z_coord']].values
        graph = generate_graph(res_coords)
        
        # 验证图节点数等于序列长度
        if graph.number_of_nodes() != len(pdb_seq):
            print(f"图节点数{graph.num_nodes()} 与序列长度{len(pdb_seq)}不一致")
            continue
        graphs.append(graph)     
        # 获取原子信息和坐标
        atom_types = atom_df['element_symbol'].tolist()  # 原子类型（C, N, O 等）
        coords = atom_df[['x_coord', 'y_coord', 'z_coord']].values  # 原子坐标
        # 存储原子和坐标信息
        test_data['atoms'].append(atom_types)
        test_data['coordinates'].append(coords)
        # 生成特征向量
        encoded_pdb_seq = seq_encode(pdb_seq).cpu().numpy()
        features = extract_features(pdb_seq, aaindex_dict)
        # encoded_pdb_seq,features = compress(encoded_pdb_seq,features)
        # normalized_encoded = scaler.fit_transform(encoded_pdb_seq.astype(np.float32))
        # normalized_features = scaler.fit_transform(features.astype(np.float32))
        combined_features = np.concatenate([encoded_pdb_seq, features], axis=-1)
        
        # 保存特征文件
        npz_path = os.path.join(root_dir, f"{sequence}.npz")
        np.savez_compressed(npz_path, wildtype_seq=combined_features)
        
        # 保存图文件
        graph_path = os.path.join(graph_dir, f"{sequence}.bin")
        dgl.save_graphs(graph_path, [graph])
        valid_indices.append(index)
        feature = [];coord_lis=[];index=0
        for letter in atom_df['letter']:
            if letter!=pdb_seq[index]:
                index += 1
            feature.append(combined_features[index])
            coord_lis.append(res_coords[index])
        if len(feature)!=len(atom_df):
            print(len(feature),len(atom_df))
            break
        test_data['features'].append(feature)
        test_data['res_coords'].append(coord_lis)
    except Exception as e:
        print(f"处理样本 {sequence} 时发生错误: {str(e)}")
        continue
dgl.save_graphs(root_dir+'/dgl_graph.bin', graphs)
# 可选: 更新数据框，仅保留有效样本
df_valid = df.loc[valid_indices].reset_index(drop=True)


In [4]:
def merge_dicts(dict1, dict2):
    result = dict1.copy()   # 复制dict1以避免修改原始数据
    for key, value in dict2.items():
        if key in result:
            # 如果键存在于结果字典中，则合并列表，并去除重复项
            result[key] = list(result[key] + value)
        else:
            # 如果键不存在于结果字典中，则直接添加
            result[key] = value
    return result

In [5]:
import numpy as np
np.save('data_raw.npy',merge_dicts(train_data,test_data))
#np.save('train_data_raw.npy', train_data)
#np.save('tamper_data.npy', test_data)

In [ ]:
import numpy as np
train_data = np.load('train_data.npy',allow_pickle=True).item()

In [ ]:
# 通过Python强制刷新缓存
import os, ctypes,torch
libc = ctypes.CDLL("libc.so.6")
libc.sync()
if hasattr(libc, 'syscall'):
    libc.syscall(ctypes.c_long(162), ctypes.c_long(1))  # sys_syncfs
if torch.cuda.is_available():
    torch.cuda.empty_cache()